## This code reads all the classified RSAT-2 images for Coronation Gulf between 2008 and 2020 and creates weekly mosaics of the mean value of overlaping pixels and the value closest to the valid date of the weekly CIS regional charts (Mondays at 6PM UTC)

Note: all RSAT-2 outputs have been projected to the EASE-GRID 2 projection (EPSG:6931), i.e. all pixels have the same size which was set to the pixel spacing of RSAT-2 SCWA images (50m) and are perfectly aligned.

In [ ]:
from os import walk
from os.path import join
import numpy as np
import gdal
from datetime import datetime, timedelta
import pandas as pd

Reads all the files and generates a datetime array associated with the filenames to sort the images by weeks

In [ ]:
indir = r'../../data/output'
outdir = r'../../data'

images = [join(root, f) for root, dirs, files in walk(indir) for f in files if f.endswith('.tif') and f.startswith('S0')]
dates = pd.DataFrame({'Dates':[]})
for image in images:
    
    dates = dates.append(pd.Series({'Dates':pd.to_datetime(datetime.strptime(image.split('_')[8] + image.split('_')[9], '%Y%m%d%H%M%S'))}), ignore_index = True)

dates['Filenames'] = images

del images


Initiates the first week and the dataframe

In [ ]:
curday = datetime(2008,7,14,23,0,0)

weeklydf = pd.DataFrame({'Date (YYYY-MM-DD)':[], '# Images': [], 'Total # of Pixels': [], 
                         'Mean # Ice Pixels': [], 'Mean Ice Concentration (%)': [],
                         'Mosaic # Ice Pixels': [], 'Mosaic Ice Concentration (%)': []})

Until the date is below July 31 2020, this code reads the images for a given week, mosaics the images, calculates the number of pixels and the total concentration for Coronation Gulf. The mosaiced outputs are saved as a gif.

In [ ]:
while curday < datetime(2020,7,31):

    print(curday.strftime('%Y-%m-%d'))
    
    tempdates = dates.loc[(dates.Dates <= curday) & (dates.Dates > curday - timedelta(7))].reset_index()
    tempdates.sort_values(by=['Dates'])
    
    if not tempdates.Filenames.empty:
        
        
        
        for filename in tempdates.Filenames:
            
            if tempdates.Filenames[tempdates.Filenames == filename].index == 0:
                
                tempimg = gdal.Open(join(indir, filename))
                temparray = tempimg.GetRasterBand(1).ReadAsArray()
                mosaicarray = tempimg.GetRasterBand(1).ReadAsArray()
                
            else:
                
                tempimg = None
                del tempimg
                
                tempimg = gdal.Open(join(indir, filename))
                newtemparray = tempimg.GetRasterBand(1).ReadAsArray()
                temparray = np.dstack([temparray, newtemparray])
                mosaicarray[np.isnan(newtemparray) == False] = newtemparray[np.isnan(newtemparray) == False]
                
                del newtemparray
        
        if len(temparray.shape) == 3:
        
            meanarray = np.nanmean(temparray, axis=2)
            
        else:
            
            meanarray = temparray
            
        del temparray
        
        print('Updating database...')
        
        tempseries = pd.Series({'Date (YYYY-MM-DD)': curday.strftime('%Y-%m-%d'),
                                '# Images': len(meanarray[np.isnan(meanarray)==False]),
                                'Total # of Pixels': len(meanarray[np.isnan(meanarray)==False]),
                                'Mean # Ice Pixels': len(meanarray[(np.isnan(meanarray)==False) & (meanarray<=0.5)]),
                                'Mean Ice Concentration (%)': (len(meanarray[(np.isnan(meanarray)==False) & (meanarray<=0.5)]) / len(meanarray[np.isnan(meanarray)==False]) * 100),
                                'Mosaic # Ice Pixels': len(mosaicarray[(np.isnan(mosaicarray)==False) & (mosaicarray<=0.5)]),
                                'Mosaic Ice Concentration (%)': (len(mosaicarray[(np.isnan(mosaicarray)==False) & (mosaicarray<=0.5)]) / len(mosaicarray[np.isnan(meanarray)==False]) * 100 )})
                                
        weeklydf = weeklydf.append(tempseries, ignore_index = True)
        
        print('Creating Mean Image...')
        
        driver = gdal.GetDriverByName('GTiff')
        driver.Register()    
        outmeanimg = driver.Create(join(outdir, 'WeeklyMean_' + curday.strftime('%Y%m%d') + '.tif'),
                        np.shape(meanarray)[1], np.shape(meanarray)[0], 1, gdal.GDT_Float32)
        
        geoTransform = tempimg.GetGeoTransform()
        outmeanimg.SetGeoTransform(geoTransform)
        proj = tempimg.GetProjection()
        outmeanimg.SetProjection(proj)
        
        outband = outmeanimg.GetRasterBand(1)
        outband.WriteArray(meanarray, 0, 0)
        
        outband= None
        outmeanimg = None
        driver = None
        meanarray = None
        
        del outband, outmeanimg, driver
    
        print('Creating Mosaic Image...')
    
        driver = gdal.GetDriverByName('GTiff')
        driver.Register()    
        outmosaicimg = driver.Create(join(outdir, 'WeeklyMosaic_' + curday.strftime('%Y%m%d') + '.tif'),
                        np.shape(mosaicarray)[1], np.shape(mosaicarray)[0], 1, gdal.GDT_Float32)
        
        geoTransform = tempimg.GetGeoTransform()
        outmosaicimg.SetGeoTransform(geoTransform)
        proj = tempimg.GetProjection()
        outmosaicimg.SetProjection(proj)
        
        outband = outmosaicimg.GetRasterBand(1)
        outband.WriteArray(mosaicarray, 0, 0)
        
        outband= None
        outmosaicimg = None
        driver = None
        mosaicarray = None
        tempimg = None
        
        del tempimg, outband, outmosaicimg, driver, mosaicarray, tempseries, tempdates
        
    else:
        
        print('No images for this week, updating database...')
        
        tempseries = pd.Series({'Date (YYYY-MM-DD)': curday.strftime('%Y-%m-%d'),
                                '# Images': 0,
                                'Total # of Pixels': np.nan,
                                'Mean # Ice Pixels': np.nan,
                                'Mean Ice Concentration (%)': np.nan,
                                'Mosaic # Ice Pixels': np.nan,
                                'Mosaic Ice Concentration (%)': np.nan})
                                
        weeklydf = weeklydf.append(tempseries, ignore_index = True)
        
        del tempseries, tempdates
    
    curday += timedelta(7)

Saves the dataframe as a pickle

In [ ]:
weeklydf.to_pickle(join(outdir, '../../data/WeeklySIC_RSAT-2.pkl'))